#  Real Estate Value Prediction for Different Locations

## 1. Business Understanding

### Problem Statement

MDN real estate investment firm, is loooking to find out the best locations to invest in in the future and has hired us as consultants to solve this issue

Acting as consultants for MDN real estate investment firm, our objective is to identify the best locations for their investments. Our analysis will explore the potential correlation with size rank and value of property as an indicator, helping us find the top 10 counties deemed most promising for investment, while also flagging the bottom 10 that warrant caution.

### Specific Objectives
- Identify the best locations deemed most promising for investments
- Develop a time series model that can accurately predict which locations are the best based on size rank and value of the property
- Recommend the best locations, and flag the bottom 10 that warrant caution

## 2. Data Understanding

In [35]:
#Import relevant libraries
import numpy as np
import pandas as pd

#For modeling and algorithms




In [3]:
#Import data
data = pd.read_csv('zillow_data.csv')

#Preview the data
data.head()


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [4]:
#general overview of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [5]:
#statistics of the data
data.describe()

,RegionID,RegionName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
count,14723.000000,14723.000000,14723.000000,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,...,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04
mean,81075.010052,48222.348706,7362.000000,1.182991e+05,1.184190e+05,1.185374e+05,1.186531e+05,1.187803e+05,1.189275e+05,1.191205e+05,...,2.733354e+05,2.748658e+05,2.764646e+05,2.780332e+05,2.795209e+05,2.810953e+05,2.826571e+05,2.843687e+05,2.865114e+05,2.880399e+05
std,31934.118525,29359.325439,4250.308342,8.600251e+04,8.615567e+04,8.630923e+04,8.646795e+04,8.665094e+04,8.687208e+04,8.715185e+04,...,3.603984e+05,3.614678e+05,3.627563e+05,3.644610e+05,3.656003e+05,3.670454e+05,3.695727e+05,3.717739e+05,3.724612e+05,3.720544e+05
min,58196.000000,1001.000000,1.000000,1.130000e+04,1.150000e+04,1.160000e+04,1.180000e+04,1.180000e+04,1.200000e+04,1.210000e+04,...,1.440000e+04,1.450000e+04,1.470000e+04,1.480000e+04,1.450000e+04,1.430000e+04,1.410000e+04,1.390000e+04,1.380000e+04,1.380000e+04
25%,67174.500000,22101.500000,3681.500000,6.880000e+04,6.890000e+04,6.910000e+04,6.920000e+04,6.937500e+04,6.950000e+04,6.960000e+04,...,1.269000e+05,1.275000e+05,1.282000e+05,1.287000e+05,1.292500e+05,1.299000e+05,1.306000e+05,1.310500e+05,1.319500e+05,1.324000e+05
50%,78007.000000,46106.000000,7362.000000,9.950000e+04,9.950000e+04,9.970000e+04,9.970000e+04,9.980000e+04,9.990000e+04,9.995000e+04,...,1.884000e+05,1.896000e+05,1.905000e+05,1.914000e+05,1.925000e+05,1.934000e+05,1.941000e+05,1.950000e+05,1.967000e+05,1.981000e+05
75%,90920.500000,75205.500000,11042.500000,1.432000e+05,1.433000e+05,1.432250e+05,1.432250e+05,1.435000e+05,1.437000e+05,1.439000e+05,...,3.050000e+05,3.066500e+05,3.085000e+05,3.098000e+05,3.117000e+05,3.134000e+05,3.151000e+05,3.168500e+05,3.188500e+05,3.211000e+05
max,753844.000000,99901.000000,14723.000000,3.676700e+06,3.704200e+06,3.729600e+06,3.754600e+06,3.781800e+06,3.813500e+06,3.849600e+06,...,1.888990e+07,1.870350e+07,1.860530e+07,1.856940e+07,1.842880e+07,1.830710e+07,1.836590e+07,1.853040e+07,1.833770e+07,1.789490e+07


In [6]:
# Column names
print(f"Column names: {data.columns}")

Column names: Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04'],
      dtype='object', length=272)


In [7]:
data['SizeRank'].value_counts()

2047     1
661      1
12947    1
8849     1
10896    1
        ..
3387     1
1338     1
7481     1
5432     1
2049     1
Name: SizeRank, Length: 14723, dtype: int64

In [8]:
data['CountyName'].value_counts()

Los Angeles    264
Jefferson      175
Orange         166
Washington     164
Montgomery     159
              ... 
Castro           1
Ellsworth        1
Tate             1
Luna             1
Ralls            1
Name: CountyName, Length: 1212, dtype: int64

In [9]:
len(data.City.sort_values().unique())

7554

In [10]:
data.loc[data.CountyName == 'Cabarrus']

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
111,69672,28027,Concord,NC,Charlotte,Cabarrus,112,126600.0,127300.0,127700.0,...,204000,205600,207300,209000,210400,212000,213700,215200,216700,218000
606,69670,28025,Concord,NC,Charlotte,Cabarrus,607,96500.0,97200.0,97600.0,...,156800,157500,158200,159000,159900,160800,161800,162400,162900,163300
3909,69702,28081,Kannapolis,NC,Charlotte,Cabarrus,3910,61900.0,62300.0,62600.0,...,117500,119600,121400,123200,124700,125600,125700,125800,126700,127500
4372,69704,28083,Kannapolis,NC,Charlotte,Cabarrus,4373,62100.0,62500.0,62800.0,...,111200,112400,113500,115000,117000,118600,119500,119800,120100,120700
6093,69696,28075,Harrisburg,NC,Charlotte,Cabarrus,6094,165300.0,166000.0,166400.0,...,262600,263200,264000,264600,265600,266800,268000,269100,269700,269700
9277,69720,28107,Midland,NC,Charlotte,Cabarrus,9278,95600.0,95500.0,95400.0,...,183700,182400,181700,182300,183500,184700,186700,188700,189900,190200
9791,69732,28124,Mount Pleasant,NC,Charlotte,Cabarrus,9792,71600.0,71700.0,71700.0,...,142100,142000,141400,141100,140900,140900,141100,143300,147900,152300


In [11]:
data[(data["State"] == "NY") & (data["Metro"] == "New York")]['City']

6          New York
10         New York
12         New York
13         New York
20         New York
            ...    
14513       Peconic
14530     Lewisboro
14535         Islip
14567    Orangetown
14620        Somers
Name: City, Length: 412, dtype: object

### Obseravtions
- According to the information we see that they are 272 columns and 14723 rows 
- The data in 'City' column is unclean as they are many names being used to call a city therefore we will drop it

## 3. Data Cleaning

In [42]:
# Removing unnecessary columns
# Spliting year and month

In [12]:
# Dropping unnecessary columns
data.drop(["RegionID","City","State","Metro","CountyName","SizeRank"],axis=1,inplace=True)

In [14]:
# Melting year columns to rows
CleanData = pd.melt(data, id_vars=['RegionName'], value_vars=list(data.columns[6:])).copy()
CleanData.head()

,RegionName,variable,value
0,60657,1996-09,339500.0
1,75070,1996-09,230600.0
2,77494,1996-09,205500.0
3,60614,1996-09,505700.0
4,79936,1996-09,77500.0


In [16]:
#Splitting year and month 
CleanData[["year","month"]] = CleanData["variable"].str.split(pat="-", n=-1, expand=True).copy()

In [19]:
# change datatype of year to datetime
CleanData['year'] = pd.to_datetime(CleanData['year'])

In [20]:
# drop the month column
CleanData.drop(["variable","month"],axis=1,inplace=True)

In [22]:
#Change Region name to ZipCode
CleanData.rename(index=str, columns={"RegionName": "zipcode"},inplace=True)
CleanData['zipcode']=CleanData['zipcode'].astype(str).copy()

In [27]:
CleanData['zipcode'].value_counts()

12804    260
95722    260
14206    260
20818    260
38375    260
        ... 
99623    260
47167    260
78726    260
63901    260
22967    260
Name: zipcode, Length: 14723, dtype: int64

In [23]:
CleanData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3827980 entries, 0 to 3827979
Data columns (total 3 columns):
 #   Column   Dtype         
---  ------   -----         
 0   zipcode  object        
 1   value    float64       
 2   year     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 116.8+ MB


In [30]:
#Check for missing values
CleanData.isnull().sum()

zipcode         0
value      151696
year            0
dtype: int64

In [29]:
missing_percent = (CleanData.isnull().sum() / len(CleanData)) * 100
missing_percent

zipcode    0.000000
value      3.962821
year       0.000000
dtype: float64

In [41]:
#create a variable that checks which years contain the null values
missing_values = CleanData[CleanData['value'].isnull()]

In [40]:
missing_values_by_year = missing_values.groupby('year').size()

print(missing_values_by_year)

year
1996-01-01     4156
1997-01-01    12462
1998-01-01    12432
1999-01-01    12432
2000-01-01    12432
2001-01-01    12432
2002-01-01    12432
2003-01-01    11724
2004-01-01    10518
2005-01-01     9170
2006-01-01     7842
2007-01-01     7398
2008-01-01     7230
2009-01-01     7002
2010-01-01     4402
2011-01-01     3156
2012-01-01     2580
2013-01-01     1560
2014-01-01      336
dtype: int64


In [44]:
x = CleanData.groupby('year').size()
print(x)

year
1996-01-01     58892
1997-01-01    176676
1998-01-01    176676
1999-01-01    176676
2000-01-01    176676
2001-01-01    176676
2002-01-01    176676
2003-01-01    176676
2004-01-01    176676
2005-01-01    176676
2006-01-01    176676
2007-01-01    176676
2008-01-01    176676
2009-01-01    176676
2010-01-01    176676
2011-01-01    176676
2012-01-01    176676
2013-01-01    176676
2014-01-01    176676
2015-01-01    176676
2016-01-01    176676
2017-01-01    176676
2018-01-01     58892
dtype: int64


### Observations

- the misssing vales seem to occur in the early years hence we can abondon the early years and use the data from 2015 onwords